In [1]:
import getpass
import os

Enter the OPENAI API KEY

In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

Check that the SQLite database exists

In [3]:
from langchain_community.utilities import SQLDatabase

# db = SQLDatabase.from_uri("sqlite:///Chinook.db")
# print(db.dialect)
# print(db.get_usable_table_names())
# db.run("SELECT * FROM Artist LIMIT 10;")

db = SQLDatabase.from_uri("sqlite:///pokemon.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM pokemon LIMIT 10;")

sqlite
['pokemon']


'[(0, "[\'Overgrow\', \'Chlorophyll\']", 1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 2.0, 2.0, 1.0, 0.25, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5, 49, 5120, 70, 318, \'45\', \'Seed Pokémon\', 49, 1059860, 0.7, 45, \'Fushigidaneフシギダネ\', \'Bulbasaur\', 88.1, 1, 65, 65, 45, \'grass\', \'poison\', 6.9, 1, 0), (1, "[\'Overgrow\', \'Chlorophyll\']", 1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 2.0, 2.0, 1.0, 0.25, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5, 62, 5120, 70, 405, \'45\', \'Seed Pokémon\', 63, 1059860, 1.0, 60, \'Fushigisouフシギソウ\', \'Ivysaur\', 88.1, 2, 80, 80, 60, \'grass\', \'poison\', 13.0, 1, 0), (2, "[\'Overgrow\', \'Chlorophyll\']", 1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 2.0, 2.0, 1.0, 0.25, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5, 100, 5120, 70, 625, \'45\', \'Seed Pokémon\', 123, 1059860, 2.0, 80, \'Fushigibanaフシギバナ\', \'Venusaur\', 88.1, 3, 122, 120, 80, \'grass\', \'poison\', 100.0, 1, 0), (3, "[\'Blaze\', \'Solar Power\']", 0.5, 1.0, 1.0, 1.0, 0.5, 1.0, 0.5, 1.0, 1.0, 0.5, 2.0, 0.5, 1.0, 1.0, 1.0, 2.0, 0.5, 2.0,

Using ChatGPT to translate English to SQL

In [4]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

In [5]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [6]:
chain = create_sql_query_chain(llm, db)

In [7]:
response = chain.invoke({"question": "How many pokemon are there?"})

In [8]:
response

'SELECT COUNT("index") AS total_pokemon\nFROM pokemon;'

In [9]:
db.run(response)

'[(801,)]'

In [10]:
response = chain.invoke({"question": "What kind of pokemon is pikachu?"})
response

'SELECT "name", "classfication" \nFROM pokemon \nWHERE "name" = \'Pikachu\'\nLIMIT 1;'

In [11]:
db.run(response)

"[('Pikachu', 'Mouse Pokémon')]"

In [12]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [13]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [14]:
agent_executor.invoke(
    {
        "input": "Tell me all the types of pokemon. Which one has the most weaknesses?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


pokemon
Invoking: `sql_db_schema` with `{'table_names': 'pokemon'}`



CREATE TABLE pokemon (
	"index" INTEGER, 
	abilities TEXT, 
	against_bug REAL, 
	against_dark REAL, 
	against_dragon REAL, 
	against_electric REAL, 
	against_fairy REAL, 
	against_fight REAL, 
	against_fire REAL, 
	against_flying REAL, 
	against_ghost REAL, 
	against_grass REAL, 
	against_ground REAL, 
	against_ice REAL, 
	against_normal REAL, 
	against_poison REAL, 
	against_psychic REAL, 
	against_rock REAL, 
	against_steel REAL, 
	against_water REAL, 
	attack INTEGER, 
	base_egg_steps INTEGER, 
	base_happiness INTEGER, 
	base_total INTEGER, 
	capture_rate TEXT, 
	classfication TEXT, 
	defense INTEGER, 
	experience_growth INTEGER, 
	height_m REAL, 
	hp INTEGER, 
	japanese_name TEXT, 
	name TEXT, 
	percentage_male REAL, 
	pokedex_number INTEGER, 
	sp_attack INTEGER, 
	sp_defense INTEGER, 
	speed INTEGER, 
	type1 TEXT, 
	type2 T

{'input': 'Tell me all the types of pokemon. Which one has the most weaknesses?',
 'output': 'The type of Pokemon with the most weaknesses is "water", with a total of 114 weaknesses.'}

In [15]:
agent_executor.invoke({"input": "How many generations of pokemon are there?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


pokemon
Invoking: `sql_db_schema` with `{'table_names': 'pokemon'}`



CREATE TABLE pokemon (
	"index" INTEGER, 
	abilities TEXT, 
	against_bug REAL, 
	against_dark REAL, 
	against_dragon REAL, 
	against_electric REAL, 
	against_fairy REAL, 
	against_fight REAL, 
	against_fire REAL, 
	against_flying REAL, 
	against_ghost REAL, 
	against_grass REAL, 
	against_ground REAL, 
	against_ice REAL, 
	against_normal REAL, 
	against_poison REAL, 
	against_psychic REAL, 
	against_rock REAL, 
	against_steel REAL, 
	against_water REAL, 
	attack INTEGER, 
	base_egg_steps INTEGER, 
	base_happiness INTEGER, 
	base_total INTEGER, 
	capture_rate TEXT, 
	classfication TEXT, 
	defense INTEGER, 
	experience_growth INTEGER, 
	height_m REAL, 
	hp INTEGER, 
	japanese_name TEXT, 
	name TEXT, 
	percentage_male REAL, 
	pokedex_number INTEGER, 
	sp_attack INTEGER, 
	sp_defense INTEGER, 
	speed INTEGER, 
	type1 TEXT, 
	type2 T

{'input': 'How many generations of pokemon are there?',
 'output': 'There are 7 generations of Pokemon.'}

In [16]:
agent_executor.invoke({"input": "How many pokemon are in the 1st generation?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


pokemon
Invoking: `sql_db_schema` with `{'table_names': 'pokemon'}`



CREATE TABLE pokemon (
	"index" INTEGER, 
	abilities TEXT, 
	against_bug REAL, 
	against_dark REAL, 
	against_dragon REAL, 
	against_electric REAL, 
	against_fairy REAL, 
	against_fight REAL, 
	against_fire REAL, 
	against_flying REAL, 
	against_ghost REAL, 
	against_grass REAL, 
	against_ground REAL, 
	against_ice REAL, 
	against_normal REAL, 
	against_poison REAL, 
	against_psychic REAL, 
	against_rock REAL, 
	against_steel REAL, 
	against_water REAL, 
	attack INTEGER, 
	base_egg_steps INTEGER, 
	base_happiness INTEGER, 
	base_total INTEGER, 
	capture_rate TEXT, 
	classfication TEXT, 
	defense INTEGER, 
	experience_growth INTEGER, 
	height_m REAL, 
	hp INTEGER, 
	japanese_name TEXT, 
	name TEXT, 
	percentage_male REAL, 
	pokedex_number INTEGER, 
	sp_attack INTEGER, 
	sp_defense INTEGER, 
	speed INTEGER, 
	type1 TEXT, 
	type2 T

{'input': 'How many pokemon are in the 1st generation?',
 'output': 'There are 151 Pokemon in the 1st generation.'}

In [17]:
agent_executor.invoke({"input": "Can you give a complete description of Evee from the 2nd generation game?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


pokemon
Invoking: `sql_db_schema` with `{'table_names': 'pokemon'}`



CREATE TABLE pokemon (
	"index" INTEGER, 
	abilities TEXT, 
	against_bug REAL, 
	against_dark REAL, 
	against_dragon REAL, 
	against_electric REAL, 
	against_fairy REAL, 
	against_fight REAL, 
	against_fire REAL, 
	against_flying REAL, 
	against_ghost REAL, 
	against_grass REAL, 
	against_ground REAL, 
	against_ice REAL, 
	against_normal REAL, 
	against_poison REAL, 
	against_psychic REAL, 
	against_rock REAL, 
	against_steel REAL, 
	against_water REAL, 
	attack INTEGER, 
	base_egg_steps INTEGER, 
	base_happiness INTEGER, 
	base_total INTEGER, 
	capture_rate TEXT, 
	classfication TEXT, 
	defense INTEGER, 
	experience_growth INTEGER, 
	height_m REAL, 
	hp INTEGER, 
	japanese_name TEXT, 
	name TEXT, 
	percentage_male REAL, 
	pokedex_number INTEGER, 
	sp_attack INTEGER, 
	sp_defense INTEGER, 
	speed INTEGER, 
	type1 TEXT, 
	type2 T

{'input': 'Can you give a complete description of Evee from the 2nd generation game?',
 'output': "I don't know."}

In [18]:
agent_executor.invoke({"input": "What types of pokemon are Evee strong against? Look at the 1st generation only."})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


pokemon
Invoking: `sql_db_schema` with `{'table_names': 'pokemon'}`



CREATE TABLE pokemon (
	"index" INTEGER, 
	abilities TEXT, 
	against_bug REAL, 
	against_dark REAL, 
	against_dragon REAL, 
	against_electric REAL, 
	against_fairy REAL, 
	against_fight REAL, 
	against_fire REAL, 
	against_flying REAL, 
	against_ghost REAL, 
	against_grass REAL, 
	against_ground REAL, 
	against_ice REAL, 
	against_normal REAL, 
	against_poison REAL, 
	against_psychic REAL, 
	against_rock REAL, 
	against_steel REAL, 
	against_water REAL, 
	attack INTEGER, 
	base_egg_steps INTEGER, 
	base_happiness INTEGER, 
	base_total INTEGER, 
	capture_rate TEXT, 
	classfication TEXT, 
	defense INTEGER, 
	experience_growth INTEGER, 
	height_m REAL, 
	hp INTEGER, 
	japanese_name TEXT, 
	name TEXT, 
	percentage_male REAL, 
	pokedex_number INTEGER, 
	sp_attack INTEGER, 
	sp_defense INTEGER, 
	speed INTEGER, 
	type1 TEXT, 
	type2 T

{'input': 'What types of pokemon are Evee strong against? Look at the 1st generation only.',
 'output': 'Evee from the 1st generation is strong against the following types of Pokemon: Poison, Flying, Dark, Electric, Ice, Ground, Fairy, Grass, Fighting, Psychic, Steel, Fire, Rock, and Water.'}

In [21]:
resp = agent_executor.invoke({"input": "How many columns are in the pokemon table? To the best of your ability, please tell the column name and what it means"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


pokemon
Invoking: `sql_db_schema` with `{'table_names': 'pokemon'}`



CREATE TABLE pokemon (
	"index" INTEGER, 
	abilities TEXT, 
	against_bug REAL, 
	against_dark REAL, 
	against_dragon REAL, 
	against_electric REAL, 
	against_fairy REAL, 
	against_fight REAL, 
	against_fire REAL, 
	against_flying REAL, 
	against_ghost REAL, 
	against_grass REAL, 
	against_ground REAL, 
	against_ice REAL, 
	against_normal REAL, 
	against_poison REAL, 
	against_psychic REAL, 
	against_rock REAL, 
	against_steel REAL, 
	against_water REAL, 
	attack INTEGER, 
	base_egg_steps INTEGER, 
	base_happiness INTEGER, 
	base_total INTEGER, 
	capture_rate TEXT, 
	classfication TEXT, 
	defense INTEGER, 
	experience_growth INTEGER, 
	height_m REAL, 
	hp INTEGER, 
	japanese_name TEXT, 
	name TEXT, 
	percentage_male REAL, 
	pokedex_number INTEGER, 
	sp_attack INTEGER, 
	sp_defense INTEGER, 
	speed INTEGER, 
	type1 TEXT, 
	type2 T

In [24]:
print(resp['output'])

The "pokemon" table has 41 columns. Here are some of the columns and their meanings:

1. abilities: The abilities of the Pokemon.
2. against_bug: The effectiveness of the Pokemon against Bug type.
3. against_dark: The effectiveness of the Pokemon against Dark type.
4. against_dragon: The effectiveness of the Pokemon against Dragon type.
5. against_electric: The effectiveness of the Pokemon against Electric type.
6. against_fairy: The effectiveness of the Pokemon against Fairy type.
7. against_fight: The effectiveness of the Pokemon against Fight type.
8. against_fire: The effectiveness of the Pokemon against Fire type.
9. against_flying: The effectiveness of the Pokemon against Flying type.
10. against_ghost: The effectiveness of the Pokemon against Ghost type.

These are just a few of the columns in the "pokemon" table.


In [26]:
agent_executor.invoke({"input": "Is there anything like a description column in the pokemon table? What are the kinds of things it says?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


pokemon
Invoking: `sql_db_schema` with `{'table_names': 'pokemon'}`



CREATE TABLE pokemon (
	"index" INTEGER, 
	abilities TEXT, 
	against_bug REAL, 
	against_dark REAL, 
	against_dragon REAL, 
	against_electric REAL, 
	against_fairy REAL, 
	against_fight REAL, 
	against_fire REAL, 
	against_flying REAL, 
	against_ghost REAL, 
	against_grass REAL, 
	against_ground REAL, 
	against_ice REAL, 
	against_normal REAL, 
	against_poison REAL, 
	against_psychic REAL, 
	against_rock REAL, 
	against_steel REAL, 
	against_water REAL, 
	attack INTEGER, 
	base_egg_steps INTEGER, 
	base_happiness INTEGER, 
	base_total INTEGER, 
	capture_rate TEXT, 
	classfication TEXT, 
	defense INTEGER, 
	experience_growth INTEGER, 
	height_m REAL, 
	hp INTEGER, 
	japanese_name TEXT, 
	name TEXT, 
	percentage_male REAL, 
	pokedex_number INTEGER, 
	sp_attack INTEGER, 
	sp_defense INTEGER, 
	speed INTEGER, 
	type1 TEXT, 
	type2 T

{'input': 'Is there anything like a description column in the pokemon table? What are the kinds of things it says?',
 'output': 'The `pokemon` table in the database does not have a specific column named "description." However, it does have a column named `classfication` which might contain information similar to a description. Here are some examples of the `classfication` values for the Pokemon:\n\n1. Bulbasaur - Seed Pokémon\n2. Ivysaur - Seed Pokémon\n3. Venusaur - Seed Pokémon\n\nThese values provide a brief classification or description of the Pokemon.'}